In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = "/data/ml-latest"

In [2]:
# помимо файла с описанием фильмов у нас есть файл с тегами
!head {DATA_DIR}/tags.csv

In [3]:
# создаём сессию Spark
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .getOrCreate()
)

In [4]:
import os
import pyspark.sql.functions as sql_func

# будем использовать не только теги, но и информацию о жанрах
movies = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "movies.csv"),
        header=True,
        inferSchema=True
    )
    .withColumn("genres_list", sql_func.split("genres", '\|'))
    .select("movieId", "title", "genres_list")
    .cache()
)

In [5]:
movies.limit(10).toPandas()

,movieId,title,genres_list
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]
5,6,Heat (1995),"[Action, Crime, Thriller]"
6,7,Sabrina (1995),"[Comedy, Romance]"
7,8,Tom and Huck (1995),"[Adventure, Children]"
8,9,Sudden Death (1995),[Action]
9,10,GoldenEye (1995),"[Action, Adventure, Thriller]"


In [6]:
# сначала посмотрим на общее распределение тегов
raw_tags = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "tags.csv"),
        header=True,
        inferSchema=True
    )
    .cache()
)

In [7]:
# протегрировано чуть больше половины фильмов

movie_tag_count = raw_tags.count()
tag_count = raw_tags.select("tag").distinct().count()
movie_count = raw_tags.select("movieId").distinct().count()
print("фильмов всего:", movies.select("movieId").distinct().count())
print("фильмов с тегами:", movie_count)
print("различных тегов:", tag_count)
print("всего соответствий фильм-тег:", movie_tag_count)
print("доля ненулевых элементов в матрице фильм-тег",
      movie_tag_count / movie_count / tag_count)

фильмов всего: 45843
фильмов с тегами: 25308
различных тегов: 53509
всего соответствий фильм-тег: 753170
доля ненулевых элементов в матрице фильм-тег 0.0005561710159362694


In [8]:
# некоторые теги (около шести тысяч) различаются только регистром

print(
    "различных тегов без учёта регистра:",
    raw_tags.select(sql_func.upper(sql_func.col("tag"))).distinct().count()
)

различных тегов без учёта регистра: 47952


In [9]:
# нас не будет интересовать, какой именно пользователь поставил тег
# и когда это произошло
tags = (
    raw_tags
    # теги могут различаться только регистром,
    # поэтому приведём их все к верхнему
    .select(
        sql_func.col("movieId"),
        sql_func.upper(sql_func.col("tag")).alias("tag")
    )
    .groupBy("movieId")
    .agg(sql_func.collect_list("tag").alias("tags_list"))
    .join(movies, "movieId", "right")
    .cache()
)

In [10]:
tags.limit(10).toPandas()

,movieId,tags_list,title,genres_list
0,148,"[CATCHY, NUDITY (TOPLESS), NUDITY (TOPLESS - N...","Awfully Big Adventure, An (1995)",[Drama]
1,463,None,Guilty as Sin (1993),"[Crime, Drama, Thriller]"
2,471,"[COEN BROTHERS, FISH OUT OF WATER, SATIRICAL, ...","Hudsucker Proxy, The (1994)",[Comedy]
3,496,"[LONELINESS, SUNDANCE AWARD WINNER, SUNDANCE G...",What Happened Was... (1994),"[Comedy, Drama, Romance, Thriller]"
4,833,"[SPOOF, COMEDY, HIGH SCHOOL, JON LOVITZ, LOUIS...",High School High (1996),[Comedy]
5,1088,"[CHEESY, PATRICK SWAYZE, DANCING, COMING OF AG...",Dirty Dancing (1987),"[Drama, Musical, Romance]"
6,1238,"[ABSURD, CUTE, SUPERB SOUNDTRACK, AFFECTIONATE...",Local Hero (1983),[Comedy]
7,1342,"[GRUESOME, BURNT HAIR, DEATH OF DOG, DRAMA, FA...",Candyman (1992),"[Horror, Thriller]"
8,1580,"[ACTION, ALIENS, ACTION, ALIEN, ALIENS, BUDDY ...",Men in Black (a.k.a. MIB) (1997),"[Action, Comedy, Sci-Fi]"
9,1591,"[COMIC BOOK, HELL, SUPER HERO, HORRUIBLE MOVIE...",Spawn (1997),"[Action, Adventure, Sci-Fi, Thriller]"


In [11]:
# объединим теги и жанры в единое пространство текстовых фич
from pyspark.sql.types import ArrayType, StringType

# в Spark нет некоторых полезных функций, но легко можно создавать свои
# в частности, мы хотим привести все жанры также к верхнему регистру
list_concat = sql_func.udf(
    lambda one_list, another_list:
        [str.upper(elem) for elem in one_list] + (another_list if another_list else []),
    returnType=ArrayType(StringType())
)

content_features = (
    tags
    .select(
        "movieID",
        "title",
        list_concat("genres_list", "tags_list").alias("content_features")
    )
    .cache()
)

In [12]:
content_features.limit(10).toPandas()

,movieID,title,content_features
0,148,"Awfully Big Adventure, An (1995)","[DRAMA, CATCHY, NUDITY (TOPLESS), NUDITY (TOPL..."
1,463,Guilty as Sin (1993),"[CRIME, DRAMA, THRILLER]"
2,471,"Hudsucker Proxy, The (1994)","[COMEDY, COEN BROTHERS, FISH OUT OF WATER, SAT..."
3,496,What Happened Was... (1994),"[COMEDY, DRAMA, ROMANCE, THRILLER, LONELINESS,..."
4,833,High School High (1996),"[COMEDY, SPOOF, COMEDY, HIGH SCHOOL, JON LOVIT..."
5,1088,Dirty Dancing (1987),"[DRAMA, MUSICAL, ROMANCE, CHEESY, PATRICK SWAY..."
6,1238,Local Hero (1983),"[COMEDY, ABSURD, CUTE, SUPERB SOUNDTRACK, AFFE..."
7,1342,Candyman (1992),"[HORROR, THRILLER, GRUESOME, BURNT HAIR, DEATH..."
8,1580,Men in Black (a.k.a. MIB) (1997),"[ACTION, COMEDY, SCI-FI, ACTION, ALIENS, ACTIO..."
9,1591,Spawn (1997),"[ACTION, ADVENTURE, SCI-FI, THRILLER, COMIC BO..."


In [13]:
# например, Гарри Поттер и философский камень
content_features.where("movieId == 4896").toPandas()

,movieID,title,content_features
0,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,"[ADVENTURE, CHILDREN, FANTASY, FANTASY, MAGIC,..."


In [14]:
# мешок слов (bag-of-worda)
from pyspark.sql.functions import explode, count, desc

(
    content_features
    .where("movieId == 4896")
    .select(explode("content_features").alias("words"))
    .groupBy("words")
    .agg(count("words").alias("freq"))
    .orderBy(desc("freq"))
    .toPandas()
)

,words,freq
0,MAGIC,57
1,FANTASY,52
2,HARRY POTTER,35
3,WIZARDS,31
4,BASED ON A BOOK,30
5,ADVENTURE,25
6,MYSTERY,22
7,FANTASY WORLD,17
8,GOOD VERSUS EVIL,16
9,DRAGONS,13


In [15]:
# такие фичи у нас есть для всех фильмов
content_features.count()

45843

In [16]:
# посчитаем частоты встречаемости для тегов для всех фильмов
from pyspark.ml.feature import HashingTF

term_frequencies = HashingTF(
    # от каждого тега будет вычислен хэш
    # и по факту мы будем считать частоты бакетов хэшей, а не самих тегов
    numFeatures=1024,
    inputCol="content_features",
    outputCol="term_frequencies"
).transform(content_features).cache()

In [17]:
# пример частот встречаемости бакетов (хранится в виде разреженного вектора)
term_frequencies.where("movieId == 4896").first().term_frequencies

SparseVector(1024, {23: 1.0, 27: 2.0, 29: 1.0, 33: 1.0, 37: 1.0, 55: 1.0, 60: 1.0, 61: 2.0, 73: 1.0, 98: 1.0, 103: 10.0, 113: 12.0, 117: 3.0, 121: 1.0, 124: 2.0, 126: 8.0, 128: 1.0, 180: 1.0, 189: 1.0, 192: 1.0, 200: 1.0, 232: 10.0, 236: 16.0, 239: 1.0, 255: 7.0, 258: 1.0, 264: 2.0, 272: 1.0, 285: 1.0, 303: 30.0, 307: 1.0, 309: 35.0, 344: 1.0, 373: 5.0, 383: 17.0, 386: 1.0, 389: 7.0, 395: 2.0, 398: 1.0, 404: 7.0, 409: 1.0, 414: 4.0, 451: 2.0, 462: 1.0, 466: 1.0, 473: 6.0, 489: 1.0, 492: 2.0, 499: 1.0, 507: 1.0, 513: 1.0, 518: 25.0, 521: 1.0, 530: 1.0, 535: 1.0, 547: 1.0, 548: 1.0, 553: 1.0, 569: 1.0, 596: 1.0, 606: 1.0, 609: 1.0, 611: 53.0, 647: 1.0, 665: 1.0, 678: 1.0, 683: 1.0, 684: 57.0, 687: 1.0, 702: 5.0, 711: 1.0, 715: 1.0, 716: 6.0, 718: 2.0, 726: 1.0, 729: 1.0, 742: 3.0, 743: 13.0, 750: 2.0, 752: 12.0, 753: 31.0, 761: 1.0, 785: 1.0, 790: 3.0, 827: 1.0, 832: 2.0, 838: 1.0, 857: 1.0, 870: 1.0, 878: 1.0, 881: 7.0, 887: 2.0, 888: 2.0, 892: 1.0, 897: 22.0, 914: 9.0, 923: 1.0, 926: 1

In [18]:
# теперь сделаем поправку на частоту тегов в целом, чтобы убрать неинформативные теги
# это второй шаг TF-IDF (term frequency, inverted document frequency)
from pyspark.ml.feature import IDF

idf_model = IDF(
    inputCol="term_frequencies",
    outputCol="tf_idf",
    minDocFreq=2
).fit(term_frequencies)
tf_idf = (
    idf_model
    .transform(term_frequencies)
    .select("movieId", "title", "tf_idf")
    .cache()
)

$$IDF=\ln\frac{N+1}{n+1},$$ где $N$ - общее количество документов (в нашем случае, количество фильмов), а $n$ - количество документов, в которых встречается данный терм (в нашем случае - количество фильмов, которым соответствует определённый бакет хэш-функции от тегов)

In [19]:
# собственно, сами инвертированные частоты термов
idf_model.idf[:20]

array([4.98360662, 6.08860871, 5.1495033 , 4.41764161, 5.03590612,
       5.6767538 , 5.3577212 , 5.70911909, 4.8413554 , 5.08402537,
       5.33032223, 6.06017077, 5.65159524, 5.58550513, 5.28196115,
       5.62705413, 5.10537849, 5.53450258, 5.58550513, 5.33937206])

In [20]:
# результат TF-IDF преобразования для Гарри Поттера
tf_idf.where("movieId == 4896").first().tf_idf

SparseVector(1024, {23: 4.6809, 27: 9.4046, 29: 4.2332, 33: 5.54, 37: 4.8553, 55: 5.1996, 60: 5.5855, 61: 10.0718, 73: 5.3348, 98: 6.079, 103: 54.0028, 113: 57.9972, 117: 15.3596, 121: 4.1386, 124: 11.9417, 126: 42.2557, 128: 5.2441, 180: 5.084, 189: 4.7643, 192: 5.2358, 200: 5.2482, 232: 50.029, 236: 83.2571, 239: 4.9741, 255: 30.687, 258: 5.7632, 264: 12.3582, 272: 5.2777, 285: 5.5019, 303: 103.3222, 307: 4.4545, 309: 182.2641, 344: 5.54, 373: 27.2744, 383: 80.2695, 386: 4.0237, 389: 36.9437, 395: 9.3903, 398: 5.9372, 404: 33.8702, 409: 4.4213, 414: 21.394, 451: 10.5986, 462: 5.2399, 466: 6.1179, 473: 29.8826, 489: 5.9127, 492: 10.5725, 499: 4.8195, 507: 5.3348, 513: 4.3563, 518: 63.4969, 521: 5.8203, 530: 4.6304, 535: 4.5888, 547: 6.2004, 548: 4.7592, 553: 4.5427, 569: 4.8114, 596: 5.0259, 606: 4.7955, 609: 5.4755, 611: 149.6655, 647: 5.3303, 665: 4.8469, 678: 5.46, 683: 5.6271, 684: 282.4548, 687: 5.2607, 702: 20.4071, 711: 5.308, 715: 6.2332, 716: 36.361, 718: 8.7159, 726: 5.3955,

In [21]:
tf_idf.coalesce(1).write.mode("overwrite").json(os.path.join(DATA_DIR, "tf_idf.json"))

In [22]:
# Spark всегда создаёт папку при записи DataFrame на диск
!ls -lh $DATA_DIR/tf_idf.json

total 12M
-rw-r--r-- 1 root root   0 May 13 06:50 _SUCCESS
-rw-r--r-- 1 root root 12M May 13 06:50 part-00000-44f1a3f5-69fc-403f-bcb5-9164aea44878-c000.json


In [23]:
# в случае coalesce(1) в этой папке будет только один файл с данными
# и один контрольный файл
!wc $DATA_DIR/tf_idf.json/*

       0        0        0 /data/ml-latest/tf_idf.json/_SUCCESS
   45843   201006 12529653 /data/ml-latest/tf_idf.json/part-00000-44f1a3f5-69fc-403f-bcb5-9164aea44878-c000.json
   45843   201006 12529653 total


In [24]:
# в файле с данными построчно хранятся человек-читаемые JSON-преставления разреженных TF-IDF векторов
!head -1 $DATA_DIR/tf_idf.json/*

==> /data/ml-latest/tf_idf.json/_SUCCESS <==

==> /data/ml-latest/tf_idf.json/part-00000-44f1a3f5-69fc-403f-bcb5-9164aea44878-c000.json <==
{"movieId":148,"title":"Awfully Big Adventure, An (1995)","tf_idf":{"type":0,"size":1024,"indices":[434,437,572,865],"values":[0.8281629655111662,3.3299385165264987,6.137879757482,4.774574914586808]}}


In [25]:
# наиболее популярным форматом для сохранения на диске при работе со Spark является
# Parquet - https://parquet.apache.org/
tf_idf.write.mode("overwrite").parquet(os.path.join(DATA_DIR, "tf_idf.parquet"))

In [26]:
# файлы в формате Parquet занимают в два с лишним раза меньший объём, чем JSON
!du -h $DATA_DIR/tf_idf.parquet

5.2M	/data/ml-latest/tf_idf.parquet


In [27]:
# потому что это бинарный формат
!cat $DATA_DIR/tf_idf.parquet/* | head -2

PAR1 ��,� U� �       ���   ��   �  �  �  A  @  �  >  ,  7  m  %  �  J  ^  >	  c
  2  g  �  �  N  �    �  �  $  G  $  �  �  �  �  �  �  B  �  �    H  U  �  J  �  �  .  9  �!  �!  9g  Th  �h  \k  pl  �x  ;y  �z  ({  �|  �~  ��  ��  ��  ~�  g�  ��  h�  ��  ��  ��  ��  �  ��  '�  ��  �  '
cat: write error: Broken pipe


In [28]:
!ls $DATA_DIR/tf_idf.parquet

_SUCCESS
part-00000-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00001-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00002-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00003-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00004-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00005-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00006-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00007-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00008-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00009-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00010-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00011-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00012-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00013-b69c4d7d-be7d-4701-9ce7-f5d64ebf4fc9-c000.snappy.parquet
part-00014-b69c4d7d-be7d-4701-9ce7-f5d6